<a href="https://colab.research.google.com/github/vthylur/Weekly_Momentum_Breakout_Screener_-WMB_5-/blob/main/Weekly_Momentum_Breakout_Screener_(WMB_5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# === Step 1: Install Dependencies ===
!pip install yfinance ta tqdm --quiet

# === Step 2: Imports ===
import yfinance as yf
import pandas as pd
import ta
from tqdm import tqdm

# === Step 3: Load NSE 500 Sector Mapping ===
sector_df = pd.read_csv("/content/nse500_sector_mapping.csv")  # Adjust path if needed
sector_map = dict(zip(sector_df["Symbol"], sector_df["Sector"]))

# === Step 4: Define Ticker List ===
tickers = sector_df["Symbol"].unique().tolist()

# === Step 5: Indicator Calculation Function ===
def calculate_indicators(df):
    df = df.copy()
    df['Close'] = df['Close'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)

    # Bollinger Bands Width %
    bb_sma = df['Close'].rolling(window=20).mean()
    bb_std = df['Close'].rolling(window=20).std()
    df['BB_width_pct'] = ((bb_sma + 2*bb_std) - (bb_sma - 2*bb_std)) / df['Close']

    # MACD (Normalized)
    df['MACD'] = ta.trend.macd(df['Close'])
    df['MACD_pct'] = df['MACD'] / df['Close']
    df['MACD_avg_pct'] = df['MACD_pct'].rolling(52).mean()

    # ADX
    df['ADX'] = ta.trend.ADXIndicator(df['High'], df['Low'], df['Close']).adx()

    # ROC
    df['ROC'] = ta.momentum.roc(df['Close'])

    # Stochastic %D
    df['Stoch_d'] = ta.momentum.StochasticOscillator(df['High'], df['Low'], df['Close']).stoch_signal()

    return df

# === Step 6: Backtest ===
results = []

for symbol in tqdm(tickers, desc="Backtesting NSE500"):
    try:
        df = yf.download(symbol, start="2014-01-01", interval="1wk", auto_adjust=True, progress=False)

        if df.empty or df.shape[0] < 70:
            continue

        # Fix column names
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        df.reset_index(inplace=True)
        df.columns = [str.capitalize(c) for c in df.columns]

        df = calculate_indicators(df)
        df.dropna(inplace=True)

        if df.shape[0] < 10:
            continue

        for i in range(1, len(df) - 8):
            row = df.iloc[i]
            prev = df.iloc[i - 1]
            future = df.iloc[i + 1:i + 9]

            if pd.isna(row[['MACD_pct', 'MACD_avg_pct', 'ADX', 'ROC', 'BB_width_pct', 'Stoch_d']]).any():
                continue

            if (
                row['MACD_pct'] > row['MACD_avg_pct'] and
                row['ADX'] > 22 and
                row['BB_width_pct'] > prev['BB_width_pct'] and
                row['ROC'] > 2 and
                row['Stoch_d'] > 55 and
                len(future) == 8
            ):
                returns = [(f['Close'] - row['Close']) / row['Close'] * 100 for _, f in future.iterrows()]
                return_cols = {f"Return_W{j+1}": round(r, 2) for j, r in enumerate(returns)}

                results.append({
                    "Symbol": symbol,
                    "Sector": sector_map.get(symbol, "Unknown"),
                    "Entry_Date": row["Date"],
                    "Close": round(row["Close"], 2),
                    "MACD_pct": round(row["MACD_pct"], 4),
                    "MACD_avg_pct": round(row["MACD_avg_pct"], 4),
                    "ADX": round(row["ADX"], 2),
                    "ROC": round(row["ROC"], 2),
                    "BB_width_pct": round(row["BB_width_pct"], 4),
                    "Stoch_d": round(row["Stoch_d"], 2),
                    **return_cols
                })

    except Exception as e:
        print(f"{symbol} skipped: {e}")

# === Step 7: Save Output ===
result_df = pd.DataFrame(results)
result_df.to_csv("nse500_backtest_results.csv", index=False)
print("✅ Backtest complete. Results saved to 'nse500_backtest_results.csv'")


Backtesting NSE500:  29%|██▊       | 144/503 [01:16<03:21,  1.78it/s]ERROR:yfinance:HTTP Error 404: 
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['DUMMYRAYMN.NS']: YFTzMissingError('possibly delisted; no timezone found')
Backtesting NSE500: 100%|██████████| 503/503 [04:30<00:00,  1.86it/s]


✅ Backtest complete. Results saved to 'nse500_backtest_results.csv'
